# Trabalho de Conclusão de Curso - TCC
### Turma - DSA T212
### Aluno : Drausio Gomes dos Santos
### Orientadora: Gabrielle Maria Romeiro Lombardi
### Tema: Construção e uso de modelos de grafos para representação e análise da estrutura organizacional da administração pública federal.
## Resultados Preliminares

### Etapa 4: Construção dos Grafos
<p> Construção das representações de estruturas organizacionais em grafos.</p>

#### Importando as bibliotecas

In [1]:
!pip install igraph
!pip install pycairo
!pip install cairocffi
!pip install python-igraph

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import time
import datetime as dt
from datetime import datetime
import pickle
import igraph as ig
from igraph import *
print("igraph : ",ig.__version__)
import pandas as pd
import pytz

class Main:     
    def show_current_time(self):
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)
        return now

main = Main()
    


igraph :  0.10.1


#### Criando data frames para cada ano

In [3]:
anos = [2019,2020,2021,2022]
base_path = "C:/projetos/tcc_dsa_usp_esalq/"
dfs = []
for ano in anos:
    anoRef = str(ano)
    data_frame_orig = pd.read_csv(base_path + "dados/dou/" + anoRef + "/" + anoRef +".csv")    
    dfs.append(data_frame_orig)

df = pd.concat(dfs, ignore_index=True)
df["entidades"] = df["emissor"].str.split("/")
df["unidade_emissora"] = df["entidades"].apply(lambda x:x[len(x)-1])
df.insert(2,"unidade_superior", df["entidades"].apply(lambda x:x[0]))
df['tipo'] = df['tipo'].astype('category')
df['se_ato_assinado'] = df['responsavel'].notnull()
df['se_possui_cargo'] = df['cargo'].notnull()

### Análise do Dados

In [4]:
df_original = df.copy()
print("Checagem da consistência de ID único")
print(len(df["id"]))
print(len(df["id"].unique()))
print("-------------------------------------")
print(df["tipo"].unique())
print(df.sample(100000).info())
print(df.sample(100000).describe(datetime_is_numeric=True))


Checagem da consistência de ID único
3233684
3233684
-------------------------------------
['Decreto numerado', 'Lei', 'Alvará', 'Portaria', 'Despacho', ..., 'Extrato de Reconhecimento', 'Extrato de Carta-contrato', 'Extrato de Termo de Ajustamento de Conduta', 'Extrato de Dispensa de Chamamento Público', 'Aviso de Tomada de Subsídios']
Length: 235
Categories (235, object): ['Acordo Coletivo de Trabalho', 'Acórdão', 'Agenda Regulatória', 'Ajuste', ..., 'Súmula Administrativa', 'Termo de Autorização', 'Termo de Liberação de Operação', 'autorização de acesso']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 2651780 to 689323
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   id                100000 non-null  object  
 1   tipo              100000 non-null  category
 2   unidade_superior  100000 non-null  object  
 3   emissor           100000 non-null  object  
 4   data              1

In [5]:
pd.options.display.max_rows = 999
df2 = df_original.groupby(['tipo','se_ato_assinado']).size()
print(df2)

tipo                                                                             se_ato_assinado
Acordo Coletivo de Trabalho                                                      False                  65
                                                                                 True                   24
Acórdão                                                                          False                 177
                                                                                 True                 4051
Agenda Regulatória                                                               False                   3
                                                                                 True                   13
Ajuste                                                                           False                  56
                                                                                 True                   16
Alvará                                         

In [6]:
df

,id,tipo,unidade_superior,emissor,data,responsavel,cargo,entidades,unidade_emissora,se_ato_assinado,se_possui_cargo
0,S01012019/1020_20190112_991201162-2,Decreto numerado,Atos do Poder Judiciário,Atos do Poder Judiciário,12/01/2019,NaN,NaN,[Atos do Poder Judiciário],Atos do Poder Judiciário,False,False
1,S01012019/1020_20190116_11391945,Lei,Atos do Poder Legislativo,Atos do Poder Legislativo,16/01/2019,JAIR MESSIAS BOLSONARO,NaN,[Atos do Poder Legislativo],Atos do Poder Legislativo,True,False
2,S01012019/515_20190102_11303736,Alvará,Ministério da Segurança Pública,Ministério da Segurança Pública/Departamento d...,02/01/2019,ROSILENE GLEICE DUARTE SANTIAGO,NaN,"[Ministério da Segurança Pública, Departamento...",Coordenação-Geral de Controle de Serviços e Pr...,True,False
3,S01012019/515_20190102_11303780,Alvará,Ministério da Segurança Pública,Ministério da Segurança Pública/Departamento d...,02/01/2019,ROSILENE GLEICE DUARTE SANTIAGO,NaN,"[Ministério da Segurança Pública, Departamento...",Coordenação-Geral de Controle de Serviços e Pr...,True,False
4,S01012019/515_20190102_11303801,Portaria,"Ministério da Ciência, Tecnologia, Inovações e...","Ministério da Ciência, Tecnologia, Inovações e...",02/01/2019,VILMA DE FÁTIMA ALVARENGA FANIS,NaN,"[Ministério da Ciência, Tecnologia, Inovações ...",Coordenação-Geral de Radiodifusão Comunitária,True,False
...,...,...,...,...,...,...,...,...,...,...,...
3233679,S03112022/609_20221130_20116254,Aviso de Suspensão,Ministério da Defesa,Ministério da Defesa/Comando da Marinha/Empres...,30/11/2022,NAIR CARLA FROMENT TEIXEIRA,Pregoeira,"[Ministério da Defesa, Comando da Marinha, Emp...",Empresa Gerencial de Projetos Navais,True,True
3233680,S03112022/609_20221130_20116255,Resultado de Julgamento,Ministério da Economia,Ministério da Economia/Empresa de Tecnologia e...,30/11/2022,MARIO HENRIQUE BERNARDO DA SILVA,Pregoeiro,"[Ministério da Economia, Empresa de Tecnologia...",Empresa de Tecnologia e Informações da Previdê...,True,True
3233681,S03112022/609_20221130_20116333,Extrato de Termo Aditivo,Ministério das Comunicações,Ministério das Comunicações/Empresa Brasileira...,30/11/2022,NaN,NaN,"[Ministério das Comunicações, Empresa Brasilei...",Superintendência Estadual N3 BSB,False,False
3233682,S03112022/609_20221130_20116336,Extrato de Contrato,Ministério das Comunicações,Ministério das Comunicações/Empresa Brasileira...,30/11/2022,NaN,NaN,"[Ministério das Comunicações, Empresa Brasilei...",Superintendência Estadual N1 MG,False,False


#### Montando os grafo integrado utilizando IGraph

In [ ]:
def cria_vertice_unidade(g,nome_unidade, row, data):
    unidade_superior = row["unidade_superior"]
    if len(row['entidades']) > index+1:
        unidade_pai = row['entidades'][index+1] 
        label = nome_unidade + "["+str(unidade_superior) +"-->"+ str(unidade_pai) +"]"
    else:
        label = nome_unidade + "["+str(unidade_superior)
    
    g.add_vertices([nome_unidade])
    vert = g.vs.find(nome_unidade)            
    vert["data_inicio"]  = data
    vert["data_termino"] = data
    vert["label"] = label
    vert["weight"] =  1
    vert["tempo_medio"]=0.0 
    
def cria_vertice_assinante(g,nome_assinante, data):
    g.add_vertices([nome_assinante])
    vert = g.vs.find(nome_assinante)            
    vert["data_inicio"]  = data
    vert["data_termino"] = data
    vert["label"] = nome_assinante
    vert["weight"] =  1
    vert["tempo_medio"]=0.0 
    
def cria_aresta_assinante(g, src, targ, data):
    g.add_edges([(src , targ)])
    eg_r = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
        edge.target_vertex["name"] == targ ) 
    eg_r[0]["data_inicio"] =  data
    eg_r[0]["data_termino"] =  data
    eg_r[0]["weight"] =  1
    eg_r[0]["tempo_medio"]=0.0
    eg_r[0]["label"]=str(data) + " ["+str(eg_r[0]["weight"])+"]"

    
def atualiza_aresta_assinante(g,src,targ,data):
    eg_r = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
        edge.target_vertex["name"] == targ ) 
    if len(eg_r) > 0:                     
        eg_r[0]["weight"] = eg_r[0]["weight"] +  1
        if ( data < eg_r[0]["data_inicio"] ):
            eg_r[0]["data_inicio"] = data                    
        if ( data > eg_r[0]["data_termino"] ):
            eg_r[0]["data_termino"] = data
        eg_r[0]["tempo_medio"] = (eg_r[0]["data_termino"] - eg_r[0]["data_inicio"]) / eg_r[0]["weight"]
        eg_r[0]["label"]=str(eg_r[0]["data_inicio"]) + "-->" + str(eg_r[0]["data_termino"]) + " ["+str(eg_r[0]["weight"])+"]"
        if eg_r[0]["data_inicio"] is None:
            print("None")
    
def atualiza_vertice_unidade(vert, data):
    vert["weight"] = vert["weight"] +  1
    if ( data < vert["data_inicio"] ):
        vert["data_inicio"] = data                    
    if ( data > vert["data_termino"] ):
        vert["data_termino"] = data
    vert["tempo_medio"] = (vert["data_termino"] - vert["data_inicio"]) / vert["weight"]
    
def atualiza_aresta_unidade(g, src, targ, data) :
    eg = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
                            edge.target_vertex["name"] == targ )             
    if len(eg) > 0:                
        eg[0]["weight"] = eg[0]["weight"] +  1
        if ( data < eg[0]["data_inicio"] ):
            eg[0]["data_inicio"] = data                    
        if ( data > eg[0]["data_termino"] ):
            eg[0]["data_termino"] = data
        eg[0]["tempo_medio"] = (eg[0]["data_termino"] - eg[0]["data_inicio"]) / eg[0]["weight"]
        eg[0]["label"]=str(eg[0]["data_inicio"])+"-->"+str(eg[0]["data_termino"])
    else:
        g.add_edges([(ent , row['entidades'][index+1])])
        eg = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
                edge.target_vertex["name"] == targ ) 
        eg[0]["data_inicio"] =  data
        eg[0]["data_termino"] =  data
        eg[0]["weight"] =  1
        eg[0]["tempo_medio"]=0.0
        eg[0]["label"]=data
    

def recupera_data(data_str):
    split_data = data_str.split("/")
    dia = split_data[0]
    mes = split_data[1]
    ano = split_data[2]
    d = dt.date(int(ano),int(mes),int(dia))
    return d

def imprime_andamento(contador, tamanho, t0):
    contador = contador + 1
    limite_novo_item = tamanho / 20
    if (contador % limite_novo_item) == 0 :
        print(round(contador / tamanho * 100,1), '% (' , \
              datetime.now(pytz.timezone('America/Sao_Paulo'))-t0 ,') |', end=' ')
        #print( (datetime.now(pytz.timezone('America/Sao_Paulo'))-t0).total_seconds() / round(contador+1 / tamanho ), " hs") 
    return contador
    
print("Início da criação dos grafo integrado.")    
t0 = main.show_current_time()
print("------------------------------------") 
amostra = 200000

g = ig.Graph(directed=True)
g["name"]="Unidades Organizacionais da Administração Pública - Grafo Integrado "
df = df[df.se_ato_assinado == True]
print("tamanho dataframe: ", len(df))
df_sample = df.sample(amostra)
print("tamanho amostra: ", len(df_sample))
contador = 0
tamanho = len(df_sample)
for index, row in df_sample.iterrows():
    
    contador = imprime_andamento(contador, tamanho , t0)    
    
    for index,ent in enumerate(row["entidades"]): 
        d = recupera_data(row["data"])
        try:
            vert = g.vs.find(ent)
            atualiza_vertice_unidade(vert, d)
        except:
            cria_vertice_unidade(g, ent, row, d)
                
    for index,ent in enumerate(row["entidades"]): 
        d = recupera_data(row["data"])
        if row["se_ato_assinado"] == True:
            id = str(row["responsavel"])+"-->"+str(row["cargo"])
            try:
                vert_r = g.vs.find(id) 
                atualiza_vertice_assinante(vert_r, d)
                atualiza_aresta_assinante(g, row["unidade_emissora"], id, d)                
                
            except:                
                try:
                    cria_vertice_assinante(g, id, d)
                    cria_aresta_assinante(g, row["unidade_emissora"], id, d)                    
                except:
                    print("Erro na criação do assinante")
        
    for index,ent in enumerate(row["entidades"]): 
        if len(row['entidades']) > index+1:
            atualiza_aresta_unidade(g, ent, row['entidades'][index+1], recupera_data(row["data"]))   
print()
print("------------------------------------")            
print("Estimativa para o dataframe completo")
print( (datetime.now(pytz.timezone('America/Sao_Paulo'))-t0).total_seconds()/ len(df_sample) * len(df) /3600, " horas") 
with open("grafoIntegrado.pkl", 'wb') as pickle_file:
    pickle.dump(g, pickle_file)
    print(" ")   
    print("------------------------------------")                 
print("Término da criação dos grafos anuais.")    
main.show_current_time()            


#### Visualizando os grafos

In [ ]:
def pesquisa_por_nome(ga,nome):
    g = ig.Graph(directed=True)
    edges_indices = []
    for ea in ga.es:
        #print(nome.lower())
        #print(ea.target_vertex["name"].lower())
        #print(nome.lower() in ea.target_vertex["name"].lower())
        if nome.lower() in ea.target_vertex["name"].lower() and ea["data_inicio"] is not None:
            print(ea.source_vertex["name"],"|i ",ea["data_inicio"],"|t",ea["data_inicio"],"|q",ea["weight"],"|" ,ea.target_vertex["name"])
            if ea.index not in edges_indices:
                edges_indices.append(ea.index)
                   
    print(edges_indices)        
          
    return ga.subgraph_edges(edges_indices)  
   
out_fig_name = "grafoIntegrado.eps"
grafo_integrado = pickle.load(open("grafoIntegrado.pkl", "rb"))
visual_style = {}
visual_style["bbox"] = (1600,1800)
visual_style["margin"] = 140
visual_style["vertex_color"] = 'green'
visual_style["vertex_frame_color"] = 'green'
visual_style["vertex_label_dist"] = 1.5
visual_style["edge_color"] = 'yellow'
visual_style["vertex_size"] = 20
visual_style["vertex_label_size"] = 12
visual_style["vertex_label_color"] = ['navy','maroon','black','blue','red',\
                                      'brown','fuchsia','teal','purple']
visual_style["edge_curved"] = True
#visual_style["layout"] = grafo2019.layout_reingold_tilford_circular()
print(grafo_integrado.summary())
nome = "Paulo Roberto"
grafo_filtrado = pesquisa_por_nome(grafo_integrado,nome)
grafo_filtrado
plot(grafo_filtrado, out_fig_name, **visual_style)